In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
from os import listdir, getcwd
from os.path import isfile, join, exists
from math import radians,sin
from tqdm import tqdm

In [2]:
def great_circle_list(clon,clat,XLON,XLAT):
    """
    clon and clat are the target point (in degrees) to be compared to every point in XLON and XLAT
    XLON and XLAT should be the list of points that make up the front
    Out put is a list of Great Circle Distances (GCD) from each point to the target point in meters
    """
    rearth = 6371000 #meters
    clon,clat = map(radians,[clon,clat])
    XLON,XLAT = map(np.radians,[XLON,XLAT])
    
    return rearth * ( np.arccos(sin(clat) * np.sin(XLAT) + np.cos(clat) * np.cos(XLAT) * np.cos(clon - XLON)) )

def GCD_m2deg(GCD):
    rearth = 6371000 #meters
    return (GCD / (2*np.pi*rearth)) * 360

def GCD_deg2m(GCD):
    rearth = 6371000 #meters
    return (GCD / 360) * (2*np.pi*rearth)

def GetFrontCoordinates(lines):
    FRONT_coords = []
    number = 0
    while number < len(lines):
        if len(lines[number])<=2:
            number+=1
        elif lines[number].split()[0] in ['WARM','STNRY','COLD','OCFNT']:
            coordlist = [lines[number].split()[0],lines[number].split()[1]]
            if lines[number+1][0].isdigit():
                front_data = lines[number].split() + lines[number+1].split()
                number+=1
            else:
                front_data = lines[number].split()
            for coords in front_data[2:]:
                coordlist.append([int(coords[:2]),int(coords[2:])])
            FRONT_coords.append(coordlist)
            number+=1
        else:
            number+=1
            
    return FRONT_coords

def ExtractPlotData(frt):
    lats = []
    lons = []
    for co in frt[2:]:
        lats.append(co[0])
        lons.append(-1*co[1])
        
    if frt[0] == 'WARM':
        color = 'red'
    elif frt[0] == 'STNRY':
        color = 'green'
    elif frt[0] == 'COLD':
        color = 'blue'
    elif frt[0] == 'OCFNT':
        color = 'violet'
        
    return lons,lats,color,frt[0]

def GCD_Point2Front_AtTime(clon,clat,frts):
    mindist = []
    mintype = []
    for frt in frts:
        lons,lats,_,TYPE = ExtractPlotData(frt)
        gcds = GCD_m2deg(great_circle_list(clon,clat,np.array(lons),np.array(lats)))
        mindist.append(np.min(gcds))
        mintype.append(TYPE)
        # WHAT IF WE ALSO CHECK THE MID POINTS
        mlons = []
        mlats = []
        for gg in range(len(lons)-1):
            mlons.append((lons[gg]+lons[gg+1])/2)
            mlats.append((lats[gg]+lats[gg+1])/2)
        gcds = GCD_m2deg(great_circle_list(clon,clat,np.array(mlons),np.array(mlats)))
        if len(gcds) > 1:
            mindist.append(np.min(gcds))
            mintype.append(TYPE)
        
    return min(mindist), mintype[np.where(mindist==min(mindist))[0][0]]

In [3]:
# D2F = np.ones(Events.shape) * 999
# noFront = 0
# for lm in tqdm(range(len(D2F))):
#     year = pcip['time'].dt.year.values[Events[lm]]
#     month= pcip['time'].dt.month.values[Events[lm]]
#     day= pcip['time'].dt.day.values[Events[lm]]
#     hour= pcip['time'].dt.hour.values[Events[lm]]
#     p2f = 'WPC_CODSUS/lores/'+str(year)+'/KWBCCODSUS_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+'00'
#     if exists(p2f):
#         with open(p2f) as f:
#             lines = f.readlines()
#         f.close()
#         FC = GetFrontCoordinates(lines)
#         if len(FC) == 0:
#             noFront += 1
#         else:
#             D2F[lm] = GCD_Point2Front_AtTime(pcip['longitude'].values[elons[lm]],pcip['latitude'].values[elats[lm]],FC)
#     else:
#         noFront += 1
#         continue


In [4]:
nmap = np.load('../Data/ERA5_1deg_regions7_EdgesRemoved.npy')
region = 4

flist = []
for YEAR in range(2003,2011):
    flist.append('../Data/ERA5_'+str(YEAR)+'_tp_3H.nc')
pcip = xr.open_mfdataset(flist,concat_dim='time')
WD = pcip['WetDays_lt_125e-6']

In [5]:
percent = np.nanpercentile(WD,q=90,axis=0)
isPEX = np.where(WD>np.broadcast_to(percent,WD.shape),1,0)
isPEX_r = np.where(np.broadcast_to(nmap,WD.shape)==region,isPEX,0)
Events,elats,elons = np.where(isPEX_r==1)
SEASONS = WD.time.dt.season[Events]
HOURS = WD.time.dt.hour[Events]


D2F = np.ones(Events.shape) * 999
TypeOfFront = []
for lm in tqdm(range(len(D2F))):
    TypeOfFront.append(999)
noFront = 0
for lm in tqdm(range(len(D2F))):
    if Events[lm] == isPEX_r.shape[0]-1:
        continue
    else:
        year = pcip['time'].dt.year.values[Events[lm]]
        month= pcip['time'].dt.month.values[Events[lm]]
        day  = pcip['time'].dt.day.values[Events[lm]]
        hour = pcip['time'].dt.hour.values[Events[lm]]
        p2f = 'WPC_CODSUS/lores/'+str(year)+'/KWBCCODSUS_'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_'+str(hour).zfill(2)+'00'
        
        year_1 = pcip['time'].dt.year.values[Events[lm+1]]
        month_1= pcip['time'].dt.month.values[Events[lm+1]]
        day_1  = pcip['time'].dt.day.values[Events[lm+1]]
        hour_1 = pcip['time'].dt.hour.values[Events[lm+1]]
        
        p2f_1='WPC_CODSUS/lores/'+str(year_1)+'/KWBCCODSUS_'+str(year_1)+str(month_1).zfill(2)+str(day_1).zfill(2)+'_'+str(hour_1).zfill(2)+'00'
        if exists(p2f) and ([year,month,day,hour]!=[2005,3,26,28] and [year,month,day,hour]!=[2005,3,16,18] and [year,month,day,hour]!=[2005, 3, 16, 15]):
            with open(p2f) as f:
                lines = f.readlines()
            f.close()
            FC = GetFrontCoordinates(lines)
            if len(FC) == 0:
                noFront += 1
                tmp = 999
                typ = 'N/A'
            else:
                tmp,typ = GCD_Point2Front_AtTime(pcip['longitude'].values[elons[lm]],pcip['latitude'].values[elats[lm]],FC)               
        else:
            tmp = 999
            typ = 'N/A'
            
        if exists(p2f_1) and ([year,month,day,hour]!=[2005,3,26,28] and [year,month,day,hour]!=[2005,3,16,18] and [year,month,day,hour]!=[2005, 3, 16, 15]):
            with open(p2f_1) as f:
                lines = f.readlines()
            f.close()
            FC1 = GetFrontCoordinates(lines)
            if len(FC1) == 0:
                noFront += 1 
                tmp1 = 999
                typ1 = 'N/A'
            else:
                tmp1,typ1 = GCD_Point2Front_AtTime(pcip['longitude'].values[elons[lm]],pcip['latitude'].values[elats[lm]],FC1)
        else:
            tmp = 999
            typ = 'N/A'
            
        if not exists(p2f) and not exists(p2f_1):
            noFront += 1
            continue
        if tmp <= tmp1:
            D2F[lm] = tmp
            TypeOfFront[lm] = typ
        else:
            D2F[lm] = tmp1
            TypeOfFront[lm] = typ1

100%|████████████████████████████████████| 38744/38744 [04:53<00:00, 132.13it/s]


In [13]:
TOF = np.array(TypeOfFront)

'999'

In [28]:
print("% of Events Within 5 GCD of a front")
u5 = len(np.where(D2F<=5)[0])
print(round(u5 / D2F.shape[0],2)*100)
types = TOF[np.where(D2F<=5)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u5,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u5,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u5,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u5,2)*100)
print('')

print("% of Events Within 4 GCD of a front")
u4 = len(np.where(D2F<=4)[0])
print(round(u4 / D2F.shape[0],2)*100)
types = TOF[np.where(D2F<=4)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u4,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u4,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u4,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u4,2)*100)
print('')

print("% of Events Within 3 GCD of a front")
u3 = len(np.where(D2F<=3)[0])
print(round(u3 / D2F.shape[0],2)*100)
types = TOF[np.where(D2F<=3)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u3,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u3,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u3,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u3,2)*100)
print('')

print("% of Events Within 2 GCD of a front")
u2 = len(np.where(D2F<=2)[0])
print(round(u2 / D2F.shape[0],2)*100)
types = TOF[np.where(D2F<=2)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u2,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u2,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u2,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u2,2)*100)

% of Events Within 5 GCD of a front
86.0
% of Fronts that are WARM
24.0
% of Fronts that are COLD
35.0
% of Fronts that are STNRY
37.0
% of Fronts that are OCFNT
3.0

% of Events Within 4 GCD of a front
82.0
% of Fronts that are WARM
24.0
% of Fronts that are COLD
35.0
% of Fronts that are STNRY
37.0
% of Fronts that are OCFNT
3.0

% of Events Within 3 GCD of a front
73.0
% of Fronts that are WARM
23.0
% of Fronts that are COLD
36.0
% of Fronts that are STNRY
37.0
% of Fronts that are OCFNT
3.0

% of Events Within 2 GCD of a front
60.0
% of Fronts that are WARM
22.0
% of Fronts that are COLD
38.0
% of Fronts that are STNRY
37.0
% of Fronts that are OCFNT
3.0


In [29]:
print("WINTER")
print('')
inds = np.where(SEASONS=='DJF')
d2f_winter = D2F[inds]
tof_winter = TOF[inds]

print("% of Events Within 5 GCD of a front")
u5 = len(np.where(d2f_winter<=5)[0])
print(round(u5 / d2f_winter.shape[0],2)*100)
types = tof_winter[np.where(d2f_winter<=5)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u5,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u5,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u5,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u5,2)*100)
print('')

print("% of Events Within 4 GCD of a front")
u4 = len(np.where(d2f_winter<=4)[0])
print(round(u4 / d2f_winter.shape[0],2)*100)
types = tof_winter[np.where(d2f_winter<=4)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u4,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u4,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u4,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u4,2)*100)
print('')

print("% of Events Within 3 GCD of a front")
u3 = len(np.where(d2f_winter<=3)[0])
print(round(u3 / d2f_winter.shape[0],2)*100)
types = tof_winter[np.where(d2f_winter<=3)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u3,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u3,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u3,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u3,2)*100)
print('')

print("% of Events Within 2 GCD of a front")
u2 = len(np.where(d2f_winter<=2)[0])
print(round(u2 / d2f_winter.shape[0],2)*100)
types = tof_winter[np.where(d2f_winter<=2)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u2,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u2,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u2,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u2,2)*100)

WINTER

% of Events Within 5 GCD of a front
96.0
% of Fronts that are WARM
37.0
% of Fronts that are COLD
32.0
% of Fronts that are STNRY
28.000000000000004
% of Fronts that are OCFNT
4.0

% of Events Within 4 GCD of a front
92.0
% of Fronts that are WARM
36.0
% of Fronts that are COLD
32.0
% of Fronts that are STNRY
28.000000000000004
% of Fronts that are OCFNT
4.0

% of Events Within 3 GCD of a front
82.0
% of Fronts that are WARM
34.0
% of Fronts that are COLD
33.0
% of Fronts that are STNRY
28.999999999999996
% of Fronts that are OCFNT
4.0

% of Events Within 2 GCD of a front
68.0
% of Fronts that are WARM
31.0
% of Fronts that are COLD
36.0
% of Fronts that are STNRY
28.999999999999996
% of Fronts that are OCFNT
4.0


In [30]:
print("Summer")
print('')
inds = np.where(SEASONS=='JJA')
d2f_summer = D2F[inds]
tof_summer = TOF[inds]

print("% of Events Within 5 GCD of a front")
u5 = len(np.where(d2f_summer<=5)[0])
print(round(u5 / d2f_summer.shape[0],2)*100)
types = tof_summer[np.where(d2f_summer<=5)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u5,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u5,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u5,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u5,2)*100)
print('')

print("% of Events Within 4 GCD of a front")
u4 = len(np.where(d2f_summer<=4)[0])
print(round(u4 / d2f_summer.shape[0],2)*100)
types = tof_summer[np.where(d2f_summer<=4)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u4,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u4,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u4,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u4,2)*100)
print('')

print("% of Events Within 3 GCD of a front")
u3 = len(np.where(d2f_summer<=3)[0])
print(round(u3 / d2f_summer.shape[0],2)*100)
types = tof_summer[np.where(d2f_summer<=3)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u3,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u3,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u3,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u3,2)*100)
print('')

print("% of Events Within 2 GCD of a front")
u2 = len(np.where(d2f_summer<=2)[0])
print(round(u2 / d2f_summer.shape[0],2)*100)
types = tof_summer[np.where(d2f_summer<=2)[0]]
print('% of Fronts that are WARM')
print(round(len(np.where(types=='WARM')[0]) / u2,2)*100)
print('% of Fronts that are COLD')
print(round(len(np.where(types=='COLD')[0]) / u2,2)*100)
print('% of Fronts that are STNRY')
print(round(len(np.where(types=='STNRY')[0]) / u2,2)*100)
print('% of Fronts that are OCFNT')
print(round(len(np.where(types=='OCFNT')[0]) / u2,2)*100)

Summer

% of Events Within 5 GCD of a front
71.0
% of Fronts that are WARM
9.0
% of Fronts that are COLD
30.0
% of Fronts that are STNRY
61.0
% of Fronts that are OCFNT
0.0

% of Events Within 4 GCD of a front
64.0
% of Fronts that are WARM
9.0
% of Fronts that are COLD
28.999999999999996
% of Fronts that are STNRY
62.0
% of Fronts that are OCFNT
0.0

% of Events Within 3 GCD of a front
54.0
% of Fronts that are WARM
9.0
% of Fronts that are COLD
28.000000000000004
% of Fronts that are STNRY
62.0
% of Fronts that are OCFNT
0.0

% of Events Within 2 GCD of a front
43.0
% of Fronts that are WARM
10.0
% of Fronts that are COLD
28.000000000000004
% of Fronts that are STNRY
62.0
% of Fronts that are OCFNT
0.0
